In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import folium
import shapely
import pyproj
from shapely.ops import transform, Point, Polygon

## original dataset value transformation

In [4]:
# read data and transform to wgs84

nyc_gdf = gpd.read_file('nycsquirrels_clean_1.csv')
nyc_gdf = gpd.GeoDataFrame(nyc_gdf, geometry=gpd.points_from_xy(nyc_gdf.long, nyc_gdf.lat))
nyc_gdf1 = nyc_gdf.set_crs('epsg:4326')

# convert fake bools (string) into 0s and 1s

behaviors = ['approaches', 'indifferent', 'runs_from', 
             'running', 'chasing', 'climbing', 'eating', 'foraging',
             'kuks', 'quaas', 'moans', 'tail_flags', 'tail_twitches']

bool_to_int = lambda x: 1 if x == 'TRUE' else 0

behavior_to_int = pd.DataFrame({behavior: nyc_gdf1[behavior].apply(bool_to_int) for behavior in behaviors})
nyc_gdf1.update(behavior_to_int)

# convert fake floats (string) to floats

long_float = nyc_gdf1['long'].apply(lambda x: float(x))
lat_float = nyc_gdf['lat'].apply(lambda x: float(x))

nyc_gdf1 = nyc_gdf1.assign(long = nyc_gdf1['long'].apply(lambda x: float(x)), 
                           lat = nyc_gdf['lat'].apply(lambda x: float(x)))
nyc_gdf1

,long,lat,unique_squirrel_id,hectare,shift,date,hectare_squirrel_number,age,primary_fur_color,combination_of_primary_and_highlight_color,...,tail_twitches,approaches,indifferent,runs_from,lat_long,community_districts,borough_boundaries,city_council_districts,police_precincts,geometry
0,-73.957044,40.794851,37E-PM-1006-03,37E,PM,10062018,3,Adult,Gray,Gray+Cinnamon,...,0,0,0,1,POINT (-73.9570437717691 40.794850940803904),19,4,19,13,POINT (-73.95704 40.79485)
1,-73.976831,40.766718,2E-AM-1010-03,02E,AM,10102018,3,Adult,Cinnamon,Cinnamon+,...,0,0,1,0,POINT (-73.9768311751004 40.76671780725581),19,4,19,13,POINT (-73.97683 40.76672)
2,-73.975725,40.769703,5D-PM-1018-05,05D,PM,10182018,5,Juvenile,Gray,Gray+,...,0,0,0,1,POINT (-73.9757249834141 40.7697032606755),19,4,19,13,POINT (-73.97572 40.76970)
3,-73.956570,40.790256,33H-AM-1019-02,33H,AM,10192018,2,Juvenile,Gray,Gray+Cinnamon,...,0,0,0,0,POINT (-73.9565700386162 40.7902561000937),19,4,19,13,POINT (-73.95657 40.79026)
4,-73.971974,40.769305,6G-PM-1020-02,06G,PM,10202018,2,Adult,Gray,Gray+,...,0,0,1,0,POINT (-73.9719735582476 40.7693045133578),19,4,19,13,POINT (-73.97197 40.76930)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2963,-73.963943,40.790868,30B-AM-1007-04,30B,AM,10072018,4,Adult,Gray,Gray+,...,0,0,0,1,POINT (-73.9639431360458 40.7908677445466),19,4,19,13,POINT (-73.96394 40.79087)
2964,-73.970402,40.782560,19A-PM-1013-05,19A,PM,10132018,5,Adult,Gray,Gray+White,...,0,0,1,0,POINT (-73.9704015859639 40.7825600069973),19,4,19,13,POINT (-73.97040 40.78256)
2965,-73.966587,40.783678,22D-PM-1012-07,22D,PM,10122018,7,Adult,Gray,"Gray+Black, Cinnamon, White",...,0,0,1,0,POINT (-73.9665871993517 40.7836775064883),19,4,19,13,POINT (-73.96659 40.78368)
2966,-73.963994,40.789915,29B-PM-1010-02,29B,PM,10102018,2,,Gray,"Gray+Cinnamon, White",...,0,0,1,0,POINT (-73.9639941227864 40.7899152327912),19,4,19,13,POINT (-73.96399 40.78992)


## add osm data